In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from tifffile import TiffFile, imwrite
from skimage.morphology import dilation


BASE_DIR = Path(r'F:\spatial_data_test\processed')
RUN_ID = '20240917_PRISM64_3D_confocal_MouseBrain_TH_new_sample'
src_dir = BASE_DIR / f'{RUN_ID}_processed'
stc_dir = src_dir / 'stitched'
read_dir = src_dir / 'readout'

In [ ]:
def build_maxima (coordinates, shape):
    Maxima = np.zeros(shape,dtype=np.uint16)
    Maxima[coordinates['Z'],coordinates['X'],coordinates['Y']] = 1
    return Maxima

In [ ]:
kernel = np.ones((5,5,5), dtype=np.uint16)
sample_image_path = stc_dir / 'cy3.tif'
with TiffFile(sample_image_path) as tif:
    page = tif.pages[0]
    y_size = page.tags['ImageLength'].value
    x_size = page.tags['ImageWidth'].value
    z_size = len(tif.pages)
    shape = (z_size, y_size, x_size)

mapped_genes = pd.read_csv(src_dir / 'mapped_genes.csv')

## projection to tiff 3d

In [ ]:
gene_tiff_path = read_dir / 'density_tiff'
gene_tiff_path.mkdir(exist_ok=True)

gene_list = sorted([int(_) for _ in mapped_genes['Gene'].unique() if _ > 0])
for label in tqdm(gene_list, desc='Dilation'):
    image = build_maxima(mapped_genes[mapped_genes['Gene']==label], shape)
    image = dilation(image, selem=kernel)
    imwrite(gene_tiff_path / f'{label:02d}_dilated.tif', image)

## projection to tiff 3d xydownsampled

In [ ]:
gene_tiff_path = read_dir / 'density_tiff_downsample'
gene_tiff_path.mkdir(exist_ok=True)
downsample_factor = 4

intensity_downsample = mapped_genes.copy()
intensity_downsample[['X','Y']] = (intensity_downsample[['X','Y']] // downsample_factor).astype(int)

shape = (shape[0], -(-shape[1] // downsample_factor), -(-shape[2] // downsample_factor))
for label in tqdm(gene_list, desc='Dilation'):
    image = build_maxima(intensity_downsample[intensity_downsample['label']==label], shape)
    image = dilation(image, selem=kernel)
    imwrite(gene_tiff_path / f'{label:02d}_dilated.tif', image)

Dilation: 100%|██████████| 63/63 [02:37<00:00,  2.50s/it]
